In [170]:
import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import numpy as np

In [171]:
def fetch_books_data(query_params):
    """
    Fetch raw data from the Google Books API with flexible query parameters.

    :param query_params: A dictionary of query parameters (e.g., {"intitle": "Python", "inauthor": "Guido"}).
    :return: The raw JSON response from the API.
    """
    # Define the base URL for the Google Books API
    url = "https://www.googleapis.com/books/v1/volumes"

    # Dynamically build the query string from the dictionary of query parameters
    # Each key-value pair in the dictionary is formatted as "key:value"
    # The "join" function combines these pairs with spaces to create the full query string
    query = " ".join(f"{key}:{value}" for key, value in query_params.items())

    # Define the parameters for the API request
    params = {
        "q": query,  # The query string constructed above
        "maxResults": 13  # Limit the number of results returned by the API to x
    }

    try:
        # Send a GET request to the Google Books API with the specified parameters
        response = requests.get(url, params=params)

        # Check for HTTP errors; raise an exception if the response status indicates an error
        response.raise_for_status()

        # If the request is successful, return the JSON data from the API response
        return response.json()

    except requests.exceptions.RequestException as e:
        # If an error occurs during the request, print the error message
        print(f"API request error: {e}")

        # Return an empty dictionary to signify failure
        return {}


In [172]:
def format_json(data):
    """
    Format and pretty-print JSON data.

    :param data: The JSON data (as a Python dictionary).
    :return: A formatted string representation of the JSON data.
    """
    try:
        # Convert JSON data to a pretty-printed string
        return json.dumps(data, indent=4)
    except (TypeError, ValueError) as e:
        # Handle errors in case the input is not valid JSON
        print(f"Error formatting JSON: {e}")
        return "{}"



In [173]:
def json_to_dataframe(raw_data):
    """
    Transform JSON data from the Google Books API into a pandas DataFrame.

    :param json_data: Raw JSON data from the Google Books API.
    :return: A pandas DataFrame containing relevant book information.
    """
    try:
        # Extract the list of books from the JSON data
        items = raw_data.get("items", [])

        # Prepare a list to store extracted metadata
        metadata = []

        # Iterate through the books and extract relevant fields
        for item in items:
            volume_info = item.get("volumeInfo", {})
            record = {
                "ID": item.get("id", np.nan),
                "Title": volume_info.get("title", np.nan),
                "Authors": ", ".join(volume_info.get("authors", [])) if "authors" in volume_info else np.nan, #note this may return an emtpy [] not NAN
                "Publisher": volume_info.get("publisher", np.nan),
                "Page Count": volume_info.get("pageCount", np.nan),
                "Language": volume_info.get("language", np.nan),
                "Category": ", ".join(volume_info.get("categories", [])) if "categories" in volume_info else np.nan, #note this may return an emtpy [] not NAN
                "Thumbnail": volume_info.get("imageLinks", {}).get("thumbnail", np.nan), #note this may return an emtpy {} not NAN


                "ISBN_13": next(
                    (
                        identifier["identifier"]
                        for identifier in volume_info.get("industryIdentifiers", [])
                        if identifier["type"] == "ISBN_13"
                    ),
                    np.nan,
                ),
                "ISBN_10": next(
                    (
                        identifier["identifier"]
                        for identifier in volume_info.get("industryIdentifiers", [])
                        if identifier["type"] == "ISBN_10"
                    ),
                    np.nan,
                ),
            }
            metadata.append(record)

        # Create a DataFrame from the metadata
        return pd.DataFrame(metadata)

    except Exception as e:
        print(f"Error processing JSON data: {e}")
        return pd.DataFrame()


In [174]:
def select_book_from_results(books_df):
    """
    Allows the user to select a book from the results in the DataFrame,
    displaying them in batches of x <-- can be defined.

    :param books_df: Pandas DataFrame containing book data.
    :return: A df of the selected book's metadata or None if no valid selection is made.
    """
    # Check if the DataFrame is empty
    if books_df.empty:
        print("No books available to select.")
        return None  # Can be enhanced by asking to repeat

    start_index = 0  # Initial starting index for displaying books
    books2display = 4  # Adjusted to show x books per batch
    print(f"There were {len(books_df)} books found")

    while True:
        # Calculate end_index based on the start_index and books2display
        end_index = start_index + books2display
        subset_books = books_df.iloc[start_index:end_index]

        if subset_books.empty:
            print("No more books to display.")
            return None  # Stop if there are no more books to display

        print("\nAvailable books:")
        j=0
        for i, row in subset_books.iterrows():
            j+=1
            print(f"{j}: {row['Title']} by {row['Authors']}")

        # Show option to display more books only if there are more books to show
        if int(len(subset_books)) % int(books2display) == 0:
            print(f"{int(books2display) +1}: Show the next {int(books2display)} books") #only is shown when there are books to show

        print("0: Cancel selection")

        # Prompt the user to choose a book
        try:
            choice = int(input(f"\nEnter the number of the book you want to select, {books2display + 1} to show more, or 0 to cancel: "))

            if 1 <= choice <= len(subset_books):
                # Return the selected book's data as a df
                selected_book = subset_books.iloc[[choice - 1]]  # Use .to_dict() to return a dict [[]] returns a df
                print(f"\nYou selected: {selected_book['Title']} by {selected_book['Authors']}")
                return selected_book
            elif choice == (books2display + 1):
                # Show the next batch of books
                start_index += books2display
            elif choice == 0:
                print("Selection canceled.")
                return None
            else:
                print("Invalid selection. Please choose a valid option.")
        except ValueError:
            print("Invalid input. Please enter a number between 1 and 6, or 0 to cancel.")


In [175]:
def flush_stored_books_df():
    really = input("Do you want to empty the internal book storage??? then say: yes ")
    if really == "yes":
        global stored_books_df  # Modify global variable
        stored_books_df = None  # Reset to empty DataFrame
        print("Internal book storage emptied.")
    else:
        print("Nothing happened...")


In [176]:
def place_selection(selection_df):
    """
    Place a selection into the global stored_books_df, ensuring no duplicates are added.
    If 'stored_books_df' does not exist or is None, it initializes it as an empty DataFrame.

    Args:
    - selection_df (pd.DataFrame): A single-row DataFrame to be added to stored_books_df.

    Returns:
    - None: Updates the global stored_books_df.
    """
    global stored_books_df  # Access the global variable

    # Check if 'stored_books_df' exists or is None
    if "stored_books_df" not in globals() or stored_books_df is None:
        stored_books_df = pd.DataFrame()  # Instantiate an empty DataFrame


    # Check for duplicates
    if stored_books_df.empty:
        # If empty, directly assign selection_df to stored_books_df
        stored_books_df = selection_df
    else:
        # Check for duplicates by comparing with existing rows in stored_books_df
        if not stored_books_df.isin(selection_df.to_dict(orient='list')).all(axis=1).any():
            # If no duplicate, append the new row
            stored_books_df = pd.concat([stored_books_df, selection_df], ignore_index=True)

    print("Updated stored_books_df:")
    print(stored_books_df.head(5))


In [177]:
flush_stored_books_df()

Do you want to empty the internal book storage??? then say: yes yes
Internal book storage emptied.


In [178]:
# Define your search query (e.g., by title or author)
# example of serach query:
#query_params = {"intitle": "Python", "inauthor": "Guido", "isbn": "9781449355739"}
query_params = {"intitle": "Momo"}

raw_data = fetch_books_data(query_params)
book_df= json_to_dataframe(raw_data)
selection_df = select_book_from_results(book_df)
place_selection(selection_df)
#print(selection_df)
#formatted_json = format_json(raw_data)
#books_df = json_to_dataframe(raw_data)
#print(formatted_json)
#print(books_df.head())

There were 13 books found

Available books:
1: Momo en de tijdspaarders by Michael Andreas Helmuth Ende
2: Momo en de tijdspaarders by Michael Ende
3: Momo by Michael Ende
4: Momo oder Die seltsame Geschichte von den Zeit-Dieben und von dem Kind, das den Menschen die gestohlene Zeit zurückbrachte by Michael Ende
5: Show the next 4 books
0: Cancel selection

Enter the number of the book you want to select, 5 to show more, or 0 to cancel: 5

Available books:
1: Find Momo across Europe by Andrew Knapp
2: Momo by Michael
3: The Life of Momo by Mohamed Alhomsy
4: Momo, o, La extraña de los ladrones del tiempo y de la niña que devolvió el tiempo a los hombres by Michael Ende
5: Show the next 4 books
0: Cancel selection

Enter the number of the book you want to select, 5 to show more, or 0 to cancel: 2

You selected: 5    Momo
Name: Title, dtype: object by 5    Michael
Name: Authors, dtype: object
Updated stored_books_df:
             ID Title  Authors    Publisher  Page Count Language  \
5  

In [179]:
print(stored_books_df)

             ID Title  Authors    Publisher  Page Count Language  \
5  0y7lBQAAQBAJ  Momo  Michael  McSweeney's         211       en   

           Category                                          Thumbnail  \
5  Juvenile Fiction  http://books.google.com/books/content?id=0y7lB...   

         ISBN_13     ISBN_10  
5  9781938073991  1938073991  


In [110]:
flush_stored_books_df()
print(stored_books_df)

Do you want to empty the internal book storage??? then say: yes yes
Internal book storage emptied.
None


In [ ]:
# Set display options
pd.set_option('display.max_colwidth', None)  # This ensures that full content is shown in each column
pd.set_option('display.max_rows', 100)      # Set the number of rows you want to display
pd.set_option('display.max_columns', 20)    # Set the number of columns you want to display


In [ ]:
"""
old code to delete
def select_book_from_results(books_df):


    Allows the user to select a book from the results in the DataFrame,
    displaying them in batches of 5. If '6' is selected, the next batch is shown.

    :param books_df: Pandas DataFrame containing book data.
    :return: A dictionary of the selected book's metadata or None if no valid selection is made.


    # Check if the DataFrame is empty
    if books_df.empty:
        print("No books available to select.")
        return None  # Can be enhanced by asking to repeat

    start_index = 0  # Initial starting index for displaying books
    books2display = 4
    print(f"There were " + str(len(books_df)) + " books found")

    while True:
        # Display the next batch of books (5 at a time)
        end_index = start_index + int(books2display) #5
        subset_books = books_df.iloc[start_index:end_index]

        if subset_books.empty:
            print("No more books to display.")
            return None  # Stop if there are no more books to display

        print("\nAvailable books:")

        for i, row in subset_books.iterrows():
            print(f"{i + 1}: {row['Title']} by {row['Authors']}")

        if int(len(subset_books)) % int(books2display) == 0:
            print(f"{int(books2display) +1}: Show the next {int(books2display)} books") #only is shown when there are books to show

        print("0: Cancel selection")

        # Prompt the user to choose a book
        try:
            choice = int(input(f"\nEnter the number of the book you want to select, {int(books2display)} to show more, or 0 to cancel: "))
            if 1 <= choice <= len(subset_books):
                # Return the selected book's data as a dictionary
                selected_book = subset_books.iloc[[choice - 1 - start_index]]    #.to_dict() if I want a dict, wrap in [] to pass a list into iloc which returns a df instead of series
                print(f"\nYou selected: {selected_book['Title']} by {selected_book['Authors']}")
                return selected_book
            elif choice == (int(books2display)+1):
                # Show the next batch of books
                start_index += int(books2display)
            elif choice == 0:
                print("Selection canceled.")
                return None
            else:
                print("Invalid selection. Please choose a valid option.")
        except ValueError:
            print("Invalid input. Please enter a number between 1 and 6, or 0 to cancel.")

"""


In [117]:
"""
old code to be deleted
# Check if 'stored_books_df' exists, if not create it
if "stored_books_df" not in globals() or stored_books_df is None:
    stored_books_df = pd.DataFrame()  # Instantiate an empty DataFrame
print(type(stored_books_df))
print(type(selection_df))
# Check for duplicates by comparing with existing rows in stored_books_df
if stored_books_df.empty:
    # If empty, directly assign selection_df to stored_books_df
    stored_books_df = selection_df
else:
    # Check for duplicates by comparing with existing rows in stored_books_df
    if not stored_books_df.isin(selection_df.to_dict(orient='list')).all(axis=1).any():
        # If no duplicate, append the new row
        stored_books_df = pd.concat([stored_books_df, selection_df], ignore_index=True)

print(stored_books_df.head(5))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
             ID                                              Title  \
0  H9emM_LGFDEC                            Programming in Python 3   
1  KabKDAAAQBAJ  Introduction to Computation and Programming Us...   

            Authors                    Publisher  Page Count Language  \
0  Mark Summerfield  Addison-Wesley Professional         648       en   
1    John V. Guttag                    MIT Press         466       en   

    Category                                          Thumbnail  \
0  Computers  http://books.google.com/books/content?id=H9emM...   
1  Computers  http://books.google.com/books/content?id=KabKD...   

         ISBN_13     ISBN_10  
0  9780321680563  0321680561  
1  9780262529624  0262529629  
